In [ ]:
from __future__ import annotations  # Makes all type hints lazy
from typing import ClassVar, List, Union
from py2rdf.rdf_model import RDFModel, URIRefNode, MapTo

from rdflib import Literal, Namespace, URIRef, Graph
import json

ModuleNotFoundError: No module named 'rdf_model'

## Define the prefixes

In [2]:
EX_NS = Namespace("http://example.org/")

## Define the Person Class

In [ ]:
class Person(RDFModel):
    
    CLASS_URI: ClassVar[URIRef] = EX_NS.Person
        

    name: Literal | str = None
    age: Literal | int = None
    partner: URIRefNode | Person = None
    children: list[URIRefNode | Person] = None
    


    mapping: ClassVar[dict] = {
        "name": EX_NS.hasName,
        "age": EX_NS.hasAge,
        "partner": MapTo(EX_NS.hasPartner, EX_NS.hasPartner),
        "children": MapTo(EX_NS.hasChild, EX_NS.hasParent)
    }



In [4]:
from pydantic import Field

class Student(Person):
    CLASS_URI: ClassVar[URIRef] = EX_NS.Student
    student_id: Literal | str = None
    
    mapping: ClassVar[dict] = {
        "student_id": EX_NS.hasStudentID
    }

class AnotherSubclassOfStudent(Student):
    CLASS_URI: ClassVar[URIRef] = EX_NS.AnotherSubclassOfStudent
    another_property: Literal | str = None
    
    mapping: ClassVar[dict] = {
        "another_property": EX_NS.hasAnotherProperty
    }

In [5]:
from rdf_model import NodeURIMeta


NodeURIClassMapping = NodeURIMeta.node_uri_class_map
print(NodeURIClassMapping)

{rdflib.term.URIRef('http://example.org/Person'): <class '__main__.Person'>, rdflib.term.URIRef('http://example.org/Student'): <class '__main__.Student'>, rdflib.term.URIRef('http://example.org/AnotherSubclassOfStudent'): <class '__main__.AnotherSubclassOfStudent'>}


In [6]:
print(Student.mapping)

{'name': rdflib.term.URIRef('http://example.org/hasName'), 'age': rdflib.term.URIRef('http://example.org/hasAge'), 'partner': <rdf_model.MapTo object at 0x10831c140>, 'children': <rdf_model.MapTo object at 0x10ec4dca0>, 'student_id': rdflib.term.URIRef('http://example.org/hasStudentID')}


In [7]:
print(AnotherSubclassOfStudent.mapping)

{'name': rdflib.term.URIRef('http://example.org/hasName'), 'age': rdflib.term.URIRef('http://example.org/hasAge'), 'partner': <rdf_model.MapTo object at 0x10831c140>, 'children': <rdf_model.MapTo object at 0x10ec4dca0>, 'student_id': rdflib.term.URIRef('http://example.org/hasStudentID'), 'another_property': rdflib.term.URIRef('http://example.org/hasAnotherProperty')}


## Serialize to String

In [8]:
peter = Person(name="Peter", age=30, label="Peter", uri=EX_NS.Peter)

print(peter)



<Person:
	uri: http://example.org/Peter
	name: Peter
	age: 30
	label: Peter
	
>


## Serialize to RDF

In [9]:
print(peter.rdf())

@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasName "Peter"^^xsd:string .




## Serialize to JSON

In [10]:
peter_dict = peter.model_dump(exclude_none=True)
json_outut = json.dumps(peter_dict, indent=4)

print(json_outut)

#json.dumps(peter, indent=4)

{
    "class_uri": "http://example.org/Person",
    "uri": "http://example.org/Peter",
    "label": "Peter",
    "name": "Peter",
    "age": 30
}


## Testing object property and reverse property

In [11]:
lois = Person(name="Lois", age=28, label="Lois", uri=EX_NS.Lois)

chris = Person(name="Chris", age=5, label="Chris", uri=EX_NS.Chris)
meg = Person(name="Meg", age=6, label="Meg", uri=EX_NS.Meg)
stewie = Person(name="Stewie", age=1, label="Stewie", uri=EX_NS.Stewie)

peter.partner = lois
lois.partner = peter
peter.children = [chris, meg, stewie]
lois.children = [chris, meg, stewie]

In [12]:
print(peter)

<Person:
	uri: http://example.org/Peter
	name: Peter
	age: 30
	partner:
	<Person:
		uri: http://example.org/Lois
		name: Lois
		age: 28
		partner: <circular reference to http://example.org/Peter>
		children:
		<Person:
			uri: http://example.org/Chris
			name: Chris
			age: 5
			label: Chris
			
		>
		<Person:
			uri: http://example.org/Meg
			name: Meg
			age: 6
			label: Meg
			
		>
		<Person:
			uri: http://example.org/Stewie
			name: Stewie
			age: 1
			label: Stewie
			
		>
		label: Lois
		
	>
	children:
		<circular reference to http://example.org/Chris>
		<circular reference to http://example.org/Meg>
		<circular reference to http://example.org/Stewie>
	label: Peter
	
>


In [13]:
print(peter.rdf())

@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Chris a ns1:Person ;
    rdfs:label "Chris"^^xsd:string ;
    ns1:hasAge 5 ;
    ns1:hasName "Chris"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Meg a ns1:Person ;
    rdfs:label "Meg"^^xsd:string ;
    ns1:hasAge 6 ;
    ns1:hasName "Meg"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Stewie a ns1:Person ;
    rdfs:label "Stewie"^^xsd:string ;
    ns1:hasAge 1 ;
    ns1:hasName "Stewie"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Lois a ns1:Person ;
    rdfs:label "Lois"^^xsd:string ;
    ns1:hasAge 28 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:Stewie ;
    ns1:hasName "Lois"^^xsd:string ;
    ns1:hasPartner ns1:Peter .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:

### Due to circular referece, partner was excluded in the JSON. This should be handled by Pydantic

In [14]:
peter_dict = peter.model_dump(exclude_none=True, exclude=["partner"])
json_outut = json.dumps(peter_dict, indent=4)
print(json_outut)

{
    "class_uri": "http://example.org/Person",
    "uri": "http://example.org/Peter",
    "label": "Peter",
    "name": "Peter",
    "age": 30,
    "children": [
        {
            "class_uri": "http://example.org/Person",
            "uri": "http://example.org/Chris",
            "label": "Chris",
            "name": "Chris",
            "age": 5
        },
        {
            "class_uri": "http://example.org/Person",
            "uri": "http://example.org/Meg",
            "label": "Meg",
            "name": "Meg",
            "age": 6
        },
        {
            "class_uri": "http://example.org/Person",
            "uri": "http://example.org/Stewie",
            "label": "Stewie",
            "name": "Stewie",
            "age": 1
        }
    ]
}


## Deserialization

In [15]:
peter_rdf_short = """@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasName "Peter"^^xsd:string .
"""
g = Graph()
peter_short= Person.deserialize(g.parse(data=peter_rdf_short, format="turtle"), node_uri=EX_NS.Peter)[str(EX_NS.Peter)]
print(peter_short)

<Person:
	uri: http://example.org/Peter
	name: Peter
	age: 30
	label: Peter
	
>


In [16]:
peter_rdf_full = """
@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Chris a ns1:Person ;
    rdfs:label "Chris"^^xsd:string ;
    ns1:hasAge 5 ;
    ns1:hasName "Chris"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Meg a ns1:Person ;
    rdfs:label "Meg"^^xsd:string ;
    ns1:hasAge 6 ;
    ns1:hasName "Meg"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Stewie a ns1:Person ;
    rdfs:label "Stewie"^^xsd:string ;
    ns1:hasAge 1 ;
    ns1:hasName "Stewie"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Lois a ns1:Person ;
    rdfs:label "Lois"^^xsd:string ;
    ns1:hasAge 28 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:Stewie ;
    ns1:hasName "Lois"^^xsd:string ;
    ns1:hasPartner ns1:Peter .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:Stewie ;
    ns1:hasName "Peter"^^xsd:string ;
    ns1:hasPartner ns1:Lois .
"""

In [17]:
g = Graph()
peter_full = Person.deserialize(g.parse(data=peter_rdf_full, format="turtle"), node_uri=EX_NS.Peter)[str(EX_NS.Peter)]

print(peter_full)


<Person:
	uri: http://example.org/Peter
	name: Peter
	age: 30
	partner:
	<Person:
		uri: http://example.org/Lois
		name: Lois
		age: 28
		partner: <circular reference to http://example.org/Peter>
		children:
		<Person:
			uri: http://example.org/Chris
			name: Chris
			age: 5
			label: Chris
			
		>
		<Person:
			uri: http://example.org/Meg
			name: Meg
			age: 6
			label: Meg
			
		>
		<Person:
			uri: http://example.org/Stewie
			name: Stewie
			age: 1
			label: Stewie
			
		>
		label: Lois
		
	>
	children:
		<circular reference to http://example.org/Chris>
		<circular reference to http://example.org/Meg>
		<circular reference to http://example.org/Stewie>
	label: Peter
	
>


In [18]:
print(peter_full.rdf())

#peter.model_dump(exclude_none=True, exclude=["partner"])

@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Chris a ns1:Person ;
    rdfs:label "Chris"^^xsd:string ;
    ns1:hasAge 5 ;
    ns1:hasName "Chris"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Meg a ns1:Person ;
    rdfs:label "Meg"^^xsd:string ;
    ns1:hasAge 6 ;
    ns1:hasName "Meg"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Stewie a ns1:Person ;
    rdfs:label "Stewie"^^xsd:string ;
    ns1:hasAge 1 ;
    ns1:hasName "Stewie"^^xsd:string ;
    ns1:hasParent ns1:Lois,
        ns1:Peter .

ns1:Lois a ns1:Person ;
    rdfs:label "Lois"^^xsd:string ;
    ns1:hasAge 28 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:Stewie ;
    ns1:hasName "Lois"^^xsd:string ;
    ns1:hasPartner ns1:Peter .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:

### When there is no class (rdf:type) is associated with an individual (e.g., Lois and their children), an URIRefNode will be used.

In [19]:
peter_rdf_partial = """
@prefix ns1: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns1:Peter a ns1:Person ;
    rdfs:label "Peter"^^xsd:string ;
    ns1:hasAge 30 ;
    ns1:hasChild ns1:Chris,
        ns1:Meg,
        ns1:Stewie ;
    ns1:hasName "Peter"^^xsd:string ;
    ns1:hasPartner ns1:Lois .
"""

g = Graph()
peter_partial = Person.deserialize(g.parse(data=peter_rdf_partial, format="turtle"), node_uri=EX_NS.Peter)[str(EX_NS.Peter)]
print(peter_partial)


<Person:
	uri: http://example.org/Peter
	name: Peter
	age: 30
	partner: uri: http://example.org/Lois
	children:
		uri: http://example.org/Chris
		uri: http://example.org/Meg
		uri: http://example.org/Stewie
	label: Peter
	
>


In [20]:
peter_partial_dict = peter_partial.model_dump(exclude_none=True)
json_outut = json.dumps(peter_partial_dict, indent=4)
print(json_outut)

{
    "class_uri": "http://example.org/Person",
    "uri": "http://example.org/Peter",
    "label": "Peter",
    "name": "Peter",
    "age": 30,
    "partner": {
        "uri": "http://example.org/Lois"
    },
    "children": [
        {
            "uri": "http://example.org/Chris"
        },
        {
            "uri": "http://example.org/Meg"
        },
        {
            "uri": "http://example.org/Stewie"
        }
    ]
}


In [22]:
print("Person mapping:", Person.mapping)
print("Student mapping:", Student.mapping)
print("AnotherSubclassOfStudent mapping:", AnotherSubclassOfStudent.mapping)


Person mapping: {'name': rdflib.term.URIRef('http://example.org/hasName'), 'age': rdflib.term.URIRef('http://example.org/hasAge'), 'partner': <rdf_model.MapTo object at 0x10831c140>, 'children': <rdf_model.MapTo object at 0x10ec4dca0>, 'label': rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')}
Student mapping: {'name': rdflib.term.URIRef('http://example.org/hasName'), 'age': rdflib.term.URIRef('http://example.org/hasAge'), 'partner': <rdf_model.MapTo object at 0x10831c140>, 'children': <rdf_model.MapTo object at 0x10ec4dca0>, 'student_id': rdflib.term.URIRef('http://example.org/hasStudentID')}
AnotherSubclassOfStudent mapping: {'name': rdflib.term.URIRef('http://example.org/hasName'), 'age': rdflib.term.URIRef('http://example.org/hasAge'), 'partner': <rdf_model.MapTo object at 0x10831c140>, 'children': <rdf_model.MapTo object at 0x10ec4dca0>, 'student_id': rdflib.term.URIRef('http://example.org/hasStudentID'), 'another_property': rdflib.term.URIRef('http://example.org/h